## A2. Data Preprocessing – Geo Data

**Description**  
This section preprocesses geospatial and community data from multiple sources.  
It includes data loading, filtering, recoding, and cleaning steps to prepare datasets for geospatial analysis.

**Data Sources**  
1. Hospital Service Areas – Dartmouth Atlas: https://data.dartmouthatlas.org/supplemental/  
2. Area Deprivation Index – Neighborhood Atlas: https://www.neighborhoodatlas.medicine.wisc.edu/  
3. Social Vulnerability Index – CDC/ATSDR: https://www.atsdr.cdc.gov/place-health/php/svi/index.html  
4. Health Professional Shortage Areas (HPSA) and Medically Underserved Areas (MUA) – HRSA: https://data.hrsa.gov/data/download  
5. Internet access, device usage, broadband access – U.S. Census API

**Purpose**  
To prepare a merged geospatial dataset from multiple sources for downstream hospital- and region-level analysis.

**Input**
- AHA 2023 and 2024 master dataframe created from A1 (AHA20232024_master.csv)
- Zipcode & Hospital Service Area Crosswalk Data (ZipHsaHrr19.csv)
- individual state ADI files (\*_ADI_\*.csv)
- SVI file (SVI_2022_US_ZCTA.csv)
- HPSA primary health file (HPSA_PCP.shp)
- HPSA dental health file (HPSA_DH.shp)
- HPSA mental health file (HPSA_MH.shp)
- MUA file (MUA.shp")

**Output**
- combined_adi_national_by_zip5.csv
- hospital_adi_df.csv
- hospital_county_internet_df.csv
- hospital_svi_df.csv
- hospital_HPSA_MUA_score_df.csv
- AHA_master_external_data.csv


### 1. load necessary libraries 

In [ ]:
## load libraries 
# Import standard Python libraries
import getpass  
import re 
import json 
import sys  

# Import data analysis and visualization libraries
import pandas as pd 
import numpy as np  
import seaborn as sns  
import matplotlib.pyplot as plt  
import geopandas as gpd
# Import datetime utilities
from datetime import datetime, timedelta  


# Import operating system utilities
import os  


### 2 load AHA master and hospital service area crosswalk data 
1. load AHA master data 
2. load hospital service area crosswalk data 
3. link AHA master and hospital service area crosswalk data 

In [ ]:
# 2_1 load AHA master data created in A1_preprocessing_hospital_data.ipynb 
AHA_master = pd.read_csv('AHA20232024_master.csv', low_memory=False)

In [ ]:
# 2_2 load hospital service area crosswalk data from Dartmouth Atlas 
hsa_crosswalk = pd.read_csv('ZipHsaHrr19.csv')

In [ ]:
# 2_3 link AHA master and hospital service area crosswalk data 

# This section links each hospital in the AHA master dataset to its corresponding Hospital Service Area (HSA) using a ZIP code-level crosswalk. The steps include:
# - Padding ZIP codes to standard 5-digit format
# - Aggregating ZIP codes by HSA
# - Merging AHA hospital records with HSA 


# Function to standardize ZIP codes to 5-digit format
def pad_zipcode(zipcode):
    """Pad ZIP code with leading zeros to ensure 5-digit format."""
    if pd.isna(zipcode):
        return zipcode
    zipcode = str(zipcode).strip()
    zipcode = ''.join(filter(str.isdigit, zipcode))  # Keep only numeric characters
    return zipcode.zfill(5)

# Standardize ZIP code in the HSA crosswalk
hsa_crosswalk['zipcode19'] = hsa_crosswalk['zipcode19'].astype(str)
hsa_crosswalk['zipcode19'] = hsa_crosswalk['zipcode19'].str[:5].apply(pad_zipcode)

# Aggregate ZIP codes by HSA code, city, and state
hsa_aggregated = hsa_crosswalk.groupby(['hsanum', 'hsacity', 'hsastate']).agg({
    'zipcode19': lambda x: list(x)
}).reset_index()

# Ensure consistent data types for merging
hsa_crosswalk['hsanum'] = hsa_crosswalk['hsanum'].astype(int)
AHA_master['hsacode_as'] = AHA_master['hsacode_as'].fillna(0).astype(int)

# Merge AHA data with HSA metadata using HSA code
merged_data = pd.merge(
    AHA_master,
    hsa_aggregated,
    how='left',
    left_on='hsacode_as',
    right_on='hsanum'
)

### 3. load geodata 
1. Area Deprivation Index - zipcode level 
2. Social Vulnerability Index - zipcode level 
3. Healthcare Professional Shortage Area - boundary with geo-dataframe 
4. Internet Access, Broadband Access, Device Usage - county level 

In [ ]:
### 3_1 Load and Aggregate Area Deprivation Index (ADI) Data

# This section processes multiple ADI files from the Neighborhood Atlas, each corresponding to a different state.  
# For each file:
# - It identifies the ZIP code column
# - Extracts 5-digit ZIP codes
# - Aggregates national and state ADI ranks at the ZIP level using median values of each hospital's service area ZIP codes 

In [ ]:
import glob

# Function to process a single ADI file and return ZIP-level aggregations
def process_adi_file(file_path):
    print(f"Processing {os.path.basename(file_path)}...")
    
    # Read the file
    df = pd.read_csv(file_path)
    print(f"  Columns: {df.columns.tolist()}")
    
    # Check for ADI columns
    has_natrank = 'ADI_NATRANK' in df.columns
    has_staterank = 'ADI_STATERANK' in df.columns
    
    # Identify the ZIP column
    zip_column = next((col for col in df.columns if 'zip' in col.lower()), None)

    # Standardize ZIP to 5 digits
    df[zip_column] = df[zip_column].astype(str)
    df['zipcode5'] = df[zip_column].str[:5]

    # Extract state code from filename
    state_code = os.path.basename(file_path).split('_')[0]
    df['state'] = state_code

    # Convert ADI columns to numeric
    if has_natrank:
        df['ADI_NATRANK'] = pd.to_numeric(df['ADI_NATRANK'], errors='coerce')
    if has_staterank:
        df['ADI_STATERANK'] = pd.to_numeric(df['ADI_STATERANK'], errors='coerce')

    # Aggregate by 5-digit ZIP
    nat_agg = None
    state_agg = None

    if has_natrank:
        nat_agg = df.groupby(['state', 'zipcode5'])['ADI_NATRANK'].median().reset_index()
        nat_agg.rename(columns={'ADI_NATRANK': 'adi_national_median'}, inplace=True)

    if has_staterank:
        state_agg = df.groupby(['state', 'zipcode5'])['ADI_STATERANK'].median().reset_index()
        state_agg.rename(columns={'ADI_STATERANK': 'adi_state_median'}, inplace=True)


    return nat_agg, state_agg

# Get all ADI CSV files in the folder
adi_files = glob.glob('../data/ADI/*_ADI_*.csv')



In [ ]:
### 3_2 Process and Merge Area Deprivation Index (ADI) Files

# **Description**  
# This section processes multiple ADI files from the Neighborhood Atlas (one per state), standardizes ZIP codes, and aggregates national and state ADI rankings at the ZIP code level. It includes:

# - Padding and standardizing ZIP codes
# - Converting ADI rank columns to numeric
# - Aggregating median ADI scores at the ZIP level per state
# - Merging all state files into combined national and state-level ADI datasets
# - Saving the outputs for downstream geospatial analysis

# The final output includes three files:
# - `combined_adi_national_by_zip5.csv`: Median national ADI ranks per ZIP

In [ ]:

# Function to pad ZIP codes
def pad_zipcode(zipcode,digits):
    """Pad ZIP code with leading zeros to make it 5 digits"""
    if pd.isna(zipcode):
        return zipcode
    # Convert to string and remove any non-numeric characters
    zipcode = str(zipcode).strip()
    zipcode = ''.join(filter(str.isdigit, zipcode))
    # Pad with leading zeros to make it 5 digits
    return zipcode.zfill(digits)

# Get all ADI files
adi_files = glob.glob('./ADI/*_ADI_*.csv')

# Containers for results
nat_aggregated_dfs = []
state_aggregated_dfs = []

# Process each file
for file_path in adi_files:
    state_code = os.path.basename(file_path).split('_')[0]
    print(f"Processing {state_code}...")
    
    # Read the ADI file
    df = pd.read_csv(file_path)
    
    # Identify the ZIP code column
    zip_column = None
    for col in df.columns:
        if 'zip' in col.lower():
            zip_column = col
            break
    
    if not zip_column:
        print(f"  Warning: No ZIP column found in {state_code}")
        continue
    
    # Check for ADI columns
    has_natrank = 'ADI_NATRANK' in df.columns
    
    if not (has_natrank):
        print(f"  Warning: No ADI columns found in {state_code}")
        continue
    
    # Convert ZIP to string and extract first 5 digits, then pad with zeros
    df[zip_column] = df[zip_column].astype(str)
    df[zip_column] = df[zip_column].apply(pad_zipcode, digits = 9)
    df['zipcode5'] = df[zip_column].str[:5].apply(pad_zipcode, digits = 5)
    df['state'] = state_code
    
    # Process national rank data
    if has_natrank:
        # Convert to numeric
        df['ADI_NATRANK'] = pd.to_numeric(df['ADI_NATRANK'], errors='coerce')
        
        # Drop rows with null ADI values
        nat_df = df.dropna(subset=['ADI_NATRANK'])
        print(f"  National rank: {len(df) - len(nat_df)} null values dropped")
        
        # Aggregate to ZIP code level
        nat_agg = nat_df.groupby(['state', 'zipcode5'])['ADI_NATRANK'].median().reset_index()
        nat_agg.rename(columns={'ADI_NATRANK': 'adi_national_median'}, inplace=True)
        
        nat_aggregated_dfs.append(nat_agg)
        print(f"  Aggregated national rank: {nat_agg.shape[0]} ZIP codes")
    

# Combine all files
combined_nat_adi = pd.concat(nat_aggregated_dfs, ignore_index=True) if nat_aggregated_dfs else None

# Save files
if combined_nat_adi is not None:
    combined_nat_adi = combined_nat_adi.drop_duplicates(subset=['zipcode5'], keep='first')
    combined_nat_adi.to_csv('combined_adi_national_by_zip5.csv', index=False)
    print(f"\nSaved national ADI data: {combined_nat_adi.shape[0]} total ZIP codes from {combined_nat_adi['state'].nunique()} states")
    
    # Print summary statistics
    print("\nNational ADI Statistics:")
    print(combined_nat_adi['adi_national_median'].describe())
    
    # Count by state
    print("\nZIP codes per state (National Rank):")
    nat_state_counts = combined_nat_adi['state'].value_counts().sort_index()
    for state, count in nat_state_counts.items():
        print(f"  {state}: {count}")


In [ ]:
combined_nat_adi = pd.read_csv('combined_adi_national_by_zip5.csv')
combined_nat_adi['zipcode5'] = combined_nat_adi['zipcode5'].astype(str).str.zfill(5)

def calculate_hospital_adi_metrics(hospital_row, zip_adi_data):
    """
    Calculate ADI metrics for a hospital based on its service area ZIP codes
    """
    # Get ZIP codes for this hospital's service area
    service_area_zips = hospital_row['zipcode19']
    
    # Check for None, empty list, or list containing only NaN values
    if service_area_zips is None or len(service_area_zips) == 0:
        return {
            'national_adi_median': np.nan
        }
    
    # Filter out any None or NaN values if it's a list
    if isinstance(service_area_zips, list):
        service_area_zips = [zip for zip in service_area_zips if pd.notna(zip)]
        if len(service_area_zips) == 0:
            return {
                'national_adi_median': np.nan
            }
    
    # Find matching ZIP codes in ADI data
    service_area_data = zip_adi_data[zip_adi_data['zipcode5'].isin(service_area_zips)]
    
    # If no matching ZIP codes, return NaN values
    if len(service_area_data) == 0:
        return {
            'national_adi_median': np.nan
        }
    
    # Calculate median of national and state ADI ranks
    national_adi_median = service_area_data['adi_national_median'].median()

    return {
        'national_adi_median': national_adi_median
    }

# Apply function to each hospital
adi_values = []
for idx, row in merged_data.iterrows():
    try:
        values = calculate_hospital_adi_metrics(row, combined_nat_adi)
        values['id_as'] = row['id_as']
        adi_values.append(values)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        adi_values.append({
            'id_as': row['id_as'],
            'national_adi_median': np.nan,
            'state_adi_median': np.nan
        })

# Create DataFrame with ADI values
hospital_adi_df = pd.DataFrame(adi_values)


In [ ]:
hospital_adi_df.to_csv('hospital_adi_df.csv', index=False)

In [ ]:
### 3_3 Load and Aggregate Social Vulnerability Index (ADI) Data

# This section processes SVI index data from the CDC and aggregates the SVI scores at the ZIP code level for each hospital's service area


In [ ]:
svi = pd.read_csv('SVI_2022_US_ZCTA.csv')
svi['zipcode5'] = svi['FIPS'].apply(pad_zipcode, digits = 5)
svi_columns = ['zipcode5','RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4','RPL_THEMES']
svi_concise = svi[svi_columns]

In [ ]:
# This function computes Social Vulnerability Index (SVI) metrics at the hospital level based on ZIP codes in their service areas.  
# For each hospital, the code:
# - Extracts its ZIP codes
# - Matches them with SVI data
# - Calculates the median values for each SVI theme across the ZIPs

# The result is a summary of social vulnerability for the hospital’s service region.

def calculate_hospital_svi_metrics(hospital_row, zip_svi_data):
    """
    Calculate ADI metrics for a hospital based on its service area ZIP codes
    """
    # Get ZIP codes for this hospital's service area
    service_area_zips = hospital_row['zipcode19']
    
    # Check for None, empty list, or list containing only NaN values
    if service_area_zips is None or len(service_area_zips) == 0:
        return {
            'svi_themes_median': np.nan,
            'svi_theme1_median': np.nan,
            'svi_theme2_median': np.nan,
            'svi_theme3_median': np.nan,
            'svi_theme4_median': np.nan
        }
    
    # Filter out any None or NaN values if it's a list
    if isinstance(service_area_zips, list):
        service_area_zips = [zip for zip in service_area_zips if pd.notna(zip)]
        if len(service_area_zips) == 0:
            return {
                'svi_themes_median': np.nan,
                'svi_theme1_median': np.nan,
                'svi_theme2_median': np.nan,
                'svi_theme3_median': np.nan,
                'svi_theme4_median': np.nan
            }
    
    # Find matching ZIP codes in SVI data
    service_area_data = zip_svi_data[zip_svi_data['zipcode5'].isin(service_area_zips)]
    
    # If no matching ZIP codes, return NaN values
    if len(service_area_data) == 0:
        return {
            'svi_themes_median': np.nan,
            'svi_theme1_median': np.nan,
            'svi_theme2_median': np.nan,
            'svi_theme3_median': np.nan,
            'svi_theme4_median': np.nan
        }
    
    # Calculate median of SVI scores for the hospital's service area
    svi_themes_median = service_area_data['RPL_THEMES'].median()
    svi_theme1_median = service_area_data['RPL_THEME1'].median()
    svi_theme2_median = service_area_data['RPL_THEME2'].median()
    svi_theme3_median = service_area_data['RPL_THEME3'].median()
    svi_theme4_median = service_area_data['RPL_THEME4'].median()

    return {
        'svi_themes_median': svi_themes_median,
        'svi_theme1_median': svi_theme1_median,
        'svi_theme2_median': svi_theme2_median,
        'svi_theme3_median': svi_theme3_median,
        'svi_theme4_median': svi_theme4_median
    }

# Apply function to each hospital
svi_values = []
for idx, row in merged_data.iterrows():
    try:
        values = calculate_hospital_svi_metrics(row, svi)
        values['id_as'] = row['id_as']
        svi_values.append(values)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        svi_values.append({
            'id_as': row['id_as'],
            'svi_themes_median': np.nan,
            'svi_theme1_median': np.nan,
            'svi_theme2_median': np.nan,
            'svi_theme3_median': np.nan,
            'svi_theme4_median': np.nan
        })

# Create DataFrame with ADI values
hospital_svi_df = pd.DataFrame(svi_values)


In [ ]:
hospital_svi_df.to_csv('hospital_svi_df.csv', index=False)

In [ ]:
### 3_4 Retrieve and Process Census Data on Internet and Device Access

# This section uses the U.S. Census Bureau's API to retrieve ACS 5-year estimates on household technology access by county.  
# It calculates metrics such as broadband access, internet access, and device availability, which can be linked to hospital county.
# This process repeats A0_data_collection.ipynb to show how the data was processed but please refer to the A0_data_collection.ipynb for more details.


# **Variables used:**
# - `B28001_001E`: Total households (computer/device access)
# - `B28001_002E`: Households with at least one device
# - `B28001_008E`: Households with no device
# - `B28002_001E`: Total households (internet subscription)
# - `B28002_008E`: Households with broadband
# - `B28002_013E`: Households with no internet access


In [ ]:
import requests
import pandas as pd

# Set up
params = {
    "get": "NAME,B28001_001E,B28001_002E,B28001_008E,B28002_001E,B28002_008E,B28002_004E,B28002_002E,B28002_013E",
    "for": "county:*",
    "in": "state:*",
    "key": API_KEY
}

# Request data
response = requests.get(url, params=params)
data = response.json()

# Convert to DataFrame
internet_access = pd.DataFrame(data[1:], columns=data[0])

# Rename columns for clarity
internet_access = internet_access.rename(columns={
    "NAME": "County",
    "B28001_001E": "Total_Computer_Households",
    "B28001_002E": "at_least_1_device",
    "B28001_008E": "no_device",
    "B28002_001E": "Total_Internet_Households",
    "B28002_004E": "With_Broadband",
    "B28002_002E": "Internet_Access",
    "B28002_013E" : "No_Internet",

    "state": "State_Code",
    "county": "County_Code"
})

# Convert numeric columns
for col in ["Total_Computer_Households", "at_least_1_device", "no_device", "Total_Internet_Households", "With_Broadband", "No_Internet"]:
    internet_access[col] = pd.to_numeric(internet_access[col])

# Calculate percent
internet_access["Device_Percent"] = (internet_access["at_least_1_device"] / internet_access["Total_Computer_Households"] * 100).round(2)
internet_access["Broadband_Percent"] = (internet_access["With_Broadband"] / internet_access["Total_Internet_Households"] * 100).round(2)
internet_access["Internet_Percent"] = (
    (1 - internet_access["No_Internet"] / internet_access["Total_Internet_Households"]) * 100
).round(2)



In [ ]:
# Make sure both columns are strings and properly formatted
internet_access['State_Code'] = internet_access['State_Code'].astype(str).str.zfill(2)
internet_access['County_Code'] = internet_access['County_Code'].astype(str).str.zfill(3)

# Create the combined FIPS column
internet_access['fstct'] = internet_access['State_Code'] + internet_access['County_Code']
internet_access_concise = internet_access[['fstct','County','Device_Percent','Broadband_Percent','Internet_Percent']]

In [ ]:
merged_data['fcounty_as'] = merged_data['fcounty_as'].astype(str).str.zfill(5)

In [ ]:
internet_merged = merged_data.merge(internet_access_concise, left_on = 'fcounty_as', right_on = 'fstct', how = 'left')[['id_as','Device_Percent','Broadband_Percent','Internet_Percent']]

In [ ]:
internet_merged.to_csv('hospital_county_internet_df.csv', index=False)

In [ ]:
### 3_5 Load HPSA and MUA data 
# This section uses geospatial joins to link each hospital with its corresponding:
# - Primary Care Health Professional Shortage Area (HPSA)
# - Dental HPSA
# - Mental Health HPSA
# - Medically Underserved Area (MUA)

# Scores from each layer are assigned to hospitals based on whether their geocoded hospital location falls within these designated regions.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# --- Load HPSA and MUA shapefiles ---
hpsa_primary = gpd.read_file("HPSA_PCP.shp")
hpsa_dental = gpd.read_file("HPSA_DH.shp")
hpsa_mental = gpd.read_file("HPSA_MH.shp")
mua = gpd.read_file("MUA.shp")

# --- Convert AHA hospitals to GeoDataFrame using address coordinates ---
AHA_master['geometry'] = AHA_master.apply(lambda row: Point(row.long_as, row.lat_as), axis=1)
hospitals_gdf = gpd.GeoDataFrame(AHA_master, geometry="geometry", crs="EPSG:4326")

# --- Ensure all layers share the same CRS ---
hpsa_primary = hpsa_primary.to_crs("EPSG:4326")
hpsa_dental = hpsa_dental.to_crs("EPSG:4326")
hpsa_mental = hpsa_mental.to_crs("EPSG:4326")
mua = mua.to_crs("EPSG:4326")

# --- Rename score columns for clarity ---
hpsa_primary['primary_hpss'] = hpsa_primary['HpsScore']
hpsa_dental['dental_hpss'] = hpsa_dental['HpsScore']
hpsa_mental['mental_hpss'] = hpsa_mental['HpsScore']
mua['mua_score'] = mua['MuaSCORE']  # IMU: 0=worst, 100=best
mua['mua_elders_score'] = mua['PpAge6PIu']  # Elder component: 0=worst, 26=best
mua['mua_infant_score'] = mua['InfMorRtIu']  # Infant component: 0=worst, 26=best

# --- Spatial joins: assign scores to hospitals within each polygon layer ---
primary_joined = gpd.sjoin(hospitals_gdf, hpsa_primary[['primary_hpss', 'geometry']], how="left", predicate="intersects")
dental_joined = gpd.sjoin(hospitals_gdf, hpsa_dental[['dental_hpss', 'geometry']], how="left", predicate="intersects")
mental_joined = gpd.sjoin(hospitals_gdf, hpsa_mental[['mental_hpss', 'geometry']], how="left", predicate="intersects")
mua_joined = gpd.sjoin(hospitals_gdf, mua[['mua_score', 'mua_elders_score', 'mua_infant_score', 'geometry']], how="left", predicate="intersects")

# --- Replace missing scores (hospitals not within HPSA/MUA) with 0 ---
primary_joined['primary_hpss'] = primary_joined['primary_hpss'].fillna(0)
dental_joined['dental_hpss'] = dental_joined['dental_hpss'].fillna(0)
mental_joined['mental_hpss'] = mental_joined['mental_hpss'].fillna(0)
mua_joined['mua_score'] = mua_joined['mua_score'].fillna(100)
mua_joined['mua_elders_score'] = mua_joined['mua_elders_score'].fillna(20.2)
mua_joined['mua_infant_score'] = mua_joined['mua_infant_score'].fillna(26)
mua_joined['mua_shortage'] = 100 - mua_joined['mua_score']
mua_joined['mua_elders_shortage'] = 20.2 - mua_joined['mua_elders_score']
mua_joined['mua_infant_shortage'] = 26 - mua_joined['mua_infant_score']

In [ ]:
mua_columns = ['mua_score', 'mua_elders_score', 'mua_infant_score', 'mua_shortage', 'mua_elders_shortage', 'mua_infant_shortage']
mua_aggregated = mua_joined.groupby('id_as')[mua_columns].mean().reset_index()
mua_aggregated.rename(columns={
    'mua_score': 'mean_mua_score',
    'mua_elders_score': 'mean_mua_elders_score',
    'mua_infant_score': 'mean_mua_infant_score', 
    'mua_shortage': 'mean_mua_shortage',
    'mua_elders_shortage': 'mean_mua_elders_shortage',
    'mua_infant_shortage': 'mean_mua_infant_shortage'
}, inplace=True)


In [ ]:
# --- Merge all HPSA and MUA summaries into one hospital-level DataFrame ---
all_hpsa_mua = mua_aggregated

# --- Preview the combined data ---
print(all_hpsa_mua.head())

# --- Show summary statistics ---
print("\nSummary statistics:")
print(all_hpsa_mua.describe())

In [ ]:
all_hpsa_mua.to_csv('hospital_HPSA_MUA_score_df.csv', index=False)

### 4 link all data to AHA_master


In [ ]:
#link all data to AHA_master
AHA_master2 = AHA_master.copy()
AHA_master2 = AHA_master2.merge(hospital_adi_df, left_on = 'id_as', right_on = 'id_as', how = 'left')
AHA_master2 = AHA_master2.merge(hospital_svi_df, left_on = 'id_as', right_on = 'id_as', how = 'left')
AHA_master2 = AHA_master2.merge(internet_merged, left_on = 'id_as', right_on = 'id_as', how = 'left')
AHA_master2 = AHA_master2.merge(all_hpsa_mua, left_on = 'id_as', right_on = 'id_as', how = 'left')




In [ ]:
AHA_master2.to_csv('AHA_master_external_data.csv', index=False)